<a href="https://colab.research.google.com/github/bes82/Leica/blob/main/Leica_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries, download network, and load network.

This cell has three jobs. 

1.) Import useful libraries for use during the pipeline.

2.) Download the pipeline network architecture and weights from a Google Drive.

3.) Load the network for use in the pipeline.

In [1]:
#@title  <- Click here to run code. Double click this text to see code. 
print("Importing Useful Libraries...")
# Importing Useful Libraries.
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import load_model

import cv2, os
from scipy.ndimage import label
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
print("Done!")

# Clone Repository And Download Network Architecture and Weights.
print("Cloning Repository...")
! git clone https://github.com/bes82/Leica
print("Done!")
%cd Leica
print("Downloading Network Architecture And Weights...")
!gdown --id "1WvlKJ1CVMrPYq_jFgI7WaDgl1TWwL6UN"
!gdown --id "1GiJghpU-zOHAxJVutCvz0Hpf3-bwzRnx"
print("Done!")

# Load Downloaded Network Architecture And Weights.
print("Loading Network Architecture and Weights...")
network_Name_One = "Phase_1_Network.h5"
network_Name_Two = "Phase_2_Network.h5"
network_One = tf.keras.models.load_model(network_Name_One)
network_Two = tf.keras.models.load_model(network_Name_Two)
print("Done!")

Importing Useful Libraries...
Done!
Cloning Repository...
Cloning into 'Leica'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
Done!
/content/Leica
Downloading...
From: https://drive.google.com/uc?id=1WvlKJ1CVMrPYq_jFgI7WaDgl1TWwL6UN
To: /content/Leica/Phase_1_Network.h5
190MB [00:01, 144MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GiJghpU-zOHAxJVutCvz0Hpf3-bwzRnx
To: /content/Leica/Phase_2_Network.h5
283MB [00:01, 186MB/s]
Done!
Loading Network Architecture and Weights...
Done!


## Create directory to hold channels for analysis.

This cell is used to create a directory that will hold channels that we want to analyze.

IMPORTANT: Once this cell is run, you will need to manually upload channel images to the directory titled "Leica/Channel_Directory". 

If you want to re-initialize the directory, then just re-run this code after analyzing your first batch of images.

In [2]:
#@title <- Click here to run code. Double click this text to view code.
# Create A Directory To Hold Channels For Analysis.
channel_Directory = "Channel_Directory/"
if os.path.isdir(channel_Directory):
  shutil.rmtree(channel_Directory)
os.mkdir(channel_Directory)

## Channel Analysis.

This cell applies analysis to all of the channels uploaded to the "Leica/Channel_Directory" directory. After each channel is analyzed, the sRBC, WBC, other, and total counts are appended to a data frame and displayed to the user.

After all channels are analyzed, the final dataframe is displayed to the user.

In [5]:
#@title <- Click here to run code. Double click this text to view code.
# An array to hold pixel areas for all tested channels.
blob_Sizes = []
output_Data_Frame = pd.DataFrame(columns = ["File Name", "Number Of Adhered 'Cells'", "sRBC Counts", "WBC Counts", "Other", "Time (s)"])
# A function which applies a zero mean normalization to the input images.
def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

# Thresholds to be tested.
thresholds = [90]
padding = 20
# This large for loop will analyze all of the channels uploaded to the Channel Directory.
for image_Name in os.listdir(channel_Directory):
    # If statement necessary to avoid possible errors during analysis on Google Colab.
    if ".ipynb" in image_Name:
      continue
    # Start time of analysis.
    start_Time = time.time()
    print("Analyzing " + image_Name[:-4])
    start_Time = time.time()
    # Reading In Channel.
    full_Channel = plt.imread(channel_Directory + image_Name)
    # Convert grayscale image to RGB if the input channel is grayscale.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel, cv2.COLOR_GRAY2RGB)
    # Defining characteristics of the input channel.
    image_Height, image_Width, channels = np.shape(full_Channel)
    # The following if statements are used to resize the channel to have
    # dimensions which are evenly dividable by 150. To avoid as much distortion
    # as possible, we will resize either up or down in each dimensions, 
    # depending on which end the dimensions is closer to.
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    full_Channel_Resized_Borders = cv2.copyMakeBorder(full_Channel_Resized.copy(), padding, padding, padding, padding, cv2.BORDER_CONSTANT)
    # Defining characteristics of the resized input channel.
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    # Creating an array which will hold predictions.
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))

    # The following chunk of code will make predictions, and create output
    # images and mask predictions.
    x_Slider = 0
    y_Slider = 0
    # Creating an array which will hold predictions for each tile.
    output_Array = np.zeros((128,128))
    # In the following for loops, we will slide through the input channel, 
    # tile by tile, and make predictions on each tile.
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        # Sliding through all tiles in a row for each row.
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            # Resizing tile to required input size.
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]/255
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            # Normalizing the tile.
            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            output = network_One.predict(current_Tile_Normalized)

            output_Max = np.argmax(output, axis = 3)
            output_Array = output_Max[0,:,:].astype('float32')
            #print(output_Array)
            # Finding the prediction for each pixel in the tile.
            #for i in range(128):
                #for j in range(128):
                    #output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            # Resizing tile back to original size.
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    # The following for loops binarize the output, after resizing distortion.
    output_Image = np.around(output_Image).astype(int)
    #for i in range(image_Height_Resized):
        #for j in range(image_Width_Resized):
            #output_Image[i,j] = int(round(output_Image[i,j],1))

    # Defining connected pixel regions in the final image.
    blobs, number_Of_Blobs = label(output_Image == 1) # whole channel image
    properties = measure.regionprops(blobs)
    centroids = [prop.centroid for prop in properties if prop.area > 50]
    #print(len(centroids))
    sRBC = 0
    WBC = 0
    other = 0
    for i in range(len(centroids)):
        cell_Region = full_Channel_Resized_Borders[int(centroids[i][0]) - 16 + padding:int(centroids[i][0]) + 16 + padding,int(centroids[i][1]) - 16 + padding:int(centroids[i][1]) + 16 + padding,:]
        cell_Region_Resized = cv2.resize(cell_Region.copy().astype('float64'), (224,224), interpolation=cv2.INTER_LINEAR)/255
        #cell_Count = cell_Count + 1
        cell_Region_Resized_Normalized = standard_norm(cell_Region_Resized.copy())
        cell_Region_Resized_Normalized = cell_Region_Resized_Normalized[None,:,:,:]
        percentages = network_Two.predict(cell_Region_Resized_Normalized)
        prediction = np.argmax(percentages)
        if prediction == 0:
            sRBC = sRBC + 1
            continue
        elif prediction == 1:
            WBC = WBC + 1
            continue
        else:
            other = other + 1
            continue
    #print("sRBC =  " + str(sRBC))
    #print("WBC = " + str(WBC))
    #print("Other = " + str(other))
    end_Time = time.time()
    time_Change = end_Time - start_Time
    #print(time_Change)
    print("======================================")
    output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids),sRBC,WBC,other,time_Change]], columns = ["File Name", "Number Of Adhered 'Cells'", "sRBC Counts", "WBC Counts", "Other", "Time (s)"]))
    display(output_Data_Frame)
print("Final Data")
display(output_Data_Frame)

Analyzing 9-3_ch3_pselectin_1348wbc_3rbc


,File Name,Number Of Adhered 'Cells',sRBC Counts,WBC Counts,Other,Time (s)
0,9-3_ch3_pselectin_1348wbc_3rbc.jpg,1415,19,1356,40,977.118931


Analyzing 10-20 - ch1 - LN - RBC 1089


,File Name,Number Of Adhered 'Cells',sRBC Counts,WBC Counts,Other,Time (s)
0,9-3_ch3_pselectin_1348wbc_3rbc.jpg,1415,19,1356,40,977.118931
0,10-20 - ch1 - LN - RBC 1089.jpg,2045,986,104,955,1119.147041


Final Data


,File Name,Number Of Adhered 'Cells',sRBC Counts,WBC Counts,Other,Time (s)
0,9-3_ch3_pselectin_1348wbc_3rbc.jpg,1415,19,1356,40,977.118931
0,10-20 - ch1 - LN - RBC 1089.jpg,2045,986,104,955,1119.147041
